In [11]:
import pandas as pd
df = pd.read_csv('/content/sample_data/03_train.csv')

for i in ['2019', '2020', '2021', '2022', '2023', '2024']:
    del df['Policy Start Year' + '_' + i]

In [12]:
from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [13]:
%pip install catboost

In [14]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost

voting_reg = VotingRegressor(
    estimators=[
        ('lin_reg', LinearRegression()),
        ('xgboost_01', xgb.XGBRegressor(max_depth=6, n_estimators=40, random_state=42)),
        ('xgboost_02', xgb.XGBRegressor(max_depth=8, n_estimators=40, random_state=42)),
        ('catboost_01',Catboost(iterations=10, depth=8, learning_rate=1, random_state=42)),
        ('catboost_02',Catboost(iterations=20, depth=8, learning_rate=1, random_state=42)),
    ]
)
voting_reg.fit(X_train, y_train)

0:	learn: 1.0679293	total: 186ms	remaining: 1.67s
1:	learn: 1.0627742	total: 400ms	remaining: 1.6s
2:	learn: 1.0623012	total: 693ms	remaining: 1.61s
3:	learn: 1.0620290	total: 996ms	remaining: 1.49s
4:	learn: 1.0617702	total: 1.31s	remaining: 1.31s
5:	learn: 1.0612695	total: 1.62s	remaining: 1.08s
6:	learn: 1.0610946	total: 1.95s	remaining: 837ms
7:	learn: 1.0609175	total: 2.3s	remaining: 575ms
8:	learn: 1.0606936	total: 2.63s	remaining: 292ms
9:	learn: 1.0604914	total: 2.93s	remaining: 0us
0:	learn: 1.0679293	total: 405ms	remaining: 7.69s
1:	learn: 1.0627742	total: 779ms	remaining: 7.01s
2:	learn: 1.0623012	total: 1.14s	remaining: 6.48s
3:	learn: 1.0620290	total: 1.5s	remaining: 6s
4:	learn: 1.0617702	total: 1.7s	remaining: 5.09s
5:	learn: 1.0612695	total: 1.87s	remaining: 4.36s
6:	learn: 1.0610946	total: 2.04s	remaining: 3.78s
7:	learn: 1.0609175	total: 2.24s	remaining: 3.36s
8:	learn: 1.0606936	total: 2.42s	remaining: 2.96s
9:	learn: 1.0604914	total: 2.59s	remaining: 2.59s
10:	learn

VotingRegressor(estimators=[('lin_reg', LinearRegression()),
                            ('xgboost_01',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          inte...
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=8,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=40,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=42, ...)),
                            ('catboost_01',
                             <catboost.core.CatBoostRegressor object at 0x7fca811a7820>),
                            ('catboost_02',
                             <catboost.core.CatBoostRegressor object at 0x7fca811a4e20>)])

In [15]:
from sklearn.metrics import root_mean_squared_error
y_pred = voting_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0553475878243848

In [16]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [17]:
y_pred = voting_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0628817058700406

In [18]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/03_test.csv')
del test_set['Premium Amount']

for i in ['2019', '2020', '2021', '2022', '2023', '2024']:
    del test_set['Policy Start Year' + '_' + i]

y_pred = voting_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('voting_reg_05.csv', index=False)